In [1]:
import base64
import boto3
import json


def understand_video(video_s3_uri, prompt):
    # Bedrock Runtime 클라이언트 생성 (us-east-1 리전 사용)
    client = boto3.client(
        "bedrock-runtime"
    )

    # 사용할 모델 ID 지정 (Nova Pro 비전 모델)
    MODEL_ID = "amazon.nova-pro-v1:0"

    # 시스템 메시지: 모델의 역할 정의
    system = [
        {
            "text": "당신은 예술 분야의 전문가입니다. 사용자 요청에 대해서 대답해주세요."
        }
    ]

    # 사용자 메시지: 비디오와 텍스트 프롬프트를 함께 전달
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "video": {
                        "format": "mp4",  # 비디오 포맷
                        "source": {
                            "s3Location": {
                                "uri": video_s3_uri,  # S3 uri
                            }
                        }
                    }
                },
                {
                    "text": prompt  # 텍스트 프롬프트
                },
            ],
        }
    ]

    # 추론 파라미터 설정
    inference_params = {
        "maxTokens": 1024,     # 최대 생성 토큰 수
        "topP": 0.1,          # top-p 샘플링
        "topK": 10,           # top-k 샘플링
        "temperature": 0.1    # 창의성 조절 (낮을수록 결정적)
    }

    # 모델 요청 바디 구성
    request_body = {
        "schemaVersion": "messages-v1",
        "messages": messages,
        "system": system,
        "inferenceConfig": inference_params,
    }

    # 모델 호출
    response = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )

    # 응답 본문 파싱
    model_response = json.loads(response["body"].read())

    # 사용량 정보 출력 (예: 사용한 토큰 수 등)
    print("\n[Usage]-------------------")
    print(json.dumps(model_response['usage'], indent=2))

    # 생성된 텍스트만 추출 및 출력
    content_text = model_response["output"]["message"]["content"][0]["text"]
    
    return content_text


In [2]:
# 실행
video_s3_uri = "s3://nova-worksop-0324/movie.mp4"
prompt = "영상에 대해서 상세하게 설명해주세요."
result = understand_video(video_s3_uri, prompt)
print("\n[Response Content Text]-------------------")
print(result)


[Usage]-------------------
{
  "inputTokens": 2943,
  "outputTokens": 216,
  "totalTokens": 3159,
  "cacheReadInputTokenCount": 0,
  "cacheWriteInputTokenCount": 0
}

[Response Content Text]-------------------
이 영상은 두 남자가 서로 싸우는 장면을 담고 있습니다. 두 남자는 모두 검은색 옷을 입고 있으며, 싸우는 장면은 어두운 배경에서 진행됩니다. 두 남자는 서로를 때리고 차며 싸우고 있으며, 때로는 땅에 넘어지기도 합니다. 싸움은 격렬하게 진행되며, 두 남자는 서로를 공격하기 위해 다양한 동작을 사용합니다. 싸움의 결과는 영상에서 보여주지 않습니다.
